In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
#from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
import tensorflow as tf

pd.options.display.max_seq_items = 2000

In [2]:
try:
    engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/eclipse_platform_ui', echo=False)

    df_ui = pd.read_sql_table('bug_and_files', engine)
except:
    df_ui = pd.read_csv("./eclipse_platform_ui.csv")

In [3]:
df_ui.head(5)

,Unnamed: 0,id,bug_id,summary,description,bag_of_word_stemmed,summary_stemmed,description_stemmed,report_time,report_timestamp,status,commit,commit_timestamp,files
0,0,1,384108,Bug 384108 JUnit view icon no longer shows pr...,Build Identifier: Version: Juno Release Build ...,unit junit view icon longer show progress exec...,unit junit view icon longer show progress exec...,build identifi version juno releas build id up...,2012-07-03 03:39:25,1.341300e+09,resolved fixed,5da5952,1.389970e+09,bundles/org.eclipse.e4.ui.workbench/src/org/ec...
1,1,2,385394,Bug 385394 Performance issue regarding enable...,Build Identifier: Version: 4.2.0 Build id: I20...,perform issu regard enabl state handl menu con...,perform issu regard enabl state handl menu con...,build identifi version build id i20120608 hi c...,2012-07-18 07:41:06,1.342610e+09,resolved fixed,8db6c32,1.389890e+09,bundles/org.eclipse.e4.ui.workbench.renderers....
2,2,3,423588,Bug 423588 [QuickAccess] Quick Access failure...,I was working in an inner from master.\n\nWhen...,quick access quickaccess quick access failur d...,quick access quickaccess quick access failur d...,work inner master restart workspac editor open...,2013-12-09 08:48:44,1.386600e+09,resolved fixed,6750d25,1.389820e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...
3,3,4,420238,Bug 420238 [CSS] Reduce whitespace usage in t...,One of the major complains I hear is that the ...,cs reduc whitespac usag default theme one majo...,cs reduc whitespac usag default theme,one major complain hear new style wast lot spa...,2013-10-24 04:08:35,1.382600e+09,verified fixed,7710840,1.389800e+09,bundles/org.eclipse.e4.ui.workbench.renderers....
4,4,5,405216,Bug 405216 Widget disposed exception in Progr...,I have a job that sets the IProgressConstants....,widget dispos except progress info item progre...,widget dispos except progress info item progre...,job set progress constant iprogressconst actio...,2013-04-08 18:21:10,1.365460e+09,resolved fixed,470f274,1.389720e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...


In [4]:
df_ui['files'].iloc[1]

'bundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/HandledContributionItem.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/MenuManagerRenderer.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolBarContributionRecord.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolBarManagerRenderer.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolItemUpdater.java\nbundles/org.eclipse.ui.workbench/Eclipse UI/org/eclipse/ui/internal/WorkbenchWindow.java\nbundles/org.eclipse.ui.workbench/Eclipse UI/org/eclipse/ui/internal/services/EvaluationService.java'

In [5]:
df_ui.shape

(6495, 14)

In [20]:
#series_files = df_ui[['files']]
series_files = df_ui['files'].apply(lambda s: s.replace(' ', ''))
series_files = series_files.rename('files_noSpace')
series_files.iloc[1]

'bundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/HandledContributionItem.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/MenuManagerRenderer.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolBarContributionRecord.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolBarManagerRenderer.java\nbundles/org.eclipse.e4.ui.workbench.renderers.swt/src/org/eclipse/e4/ui/workbench/renderers/swt/ToolItemUpdater.java\nbundles/org.eclipse.ui.workbench/EclipseUI/org/eclipse/ui/internal/WorkbenchWindow.java\nbundles/org.eclipse.ui.workbench/EclipseUI/org/eclipse/ui/internal/services/EvaluationService.java'

In [19]:
df_ui_nospace = pd.concat([df_ui, series_files], axis=1)
df_ui_nospace.head()

,Unnamed: 0,id,bug_id,summary,description,bag_of_word_stemmed,summary_stemmed,description_stemmed,report_time,report_timestamp,status,commit,commit_timestamp,files,files_noSpace
0,0,1,384108,Bug 384108 JUnit view icon no longer shows pr...,Build Identifier: Version: Juno Release Build ...,unit junit view icon longer show progress exec...,unit junit view icon longer show progress exec...,build identifi version juno releas build id up...,2012-07-03 03:39:25,1.341300e+09,resolved fixed,5da5952,1.389970e+09,bundles/org.eclipse.e4.ui.workbench/src/org/ec...,bundles/org.eclipse.e4.ui.workbench/src/org/ec...
1,1,2,385394,Bug 385394 Performance issue regarding enable...,Build Identifier: Version: 4.2.0 Build id: I20...,perform issu regard enabl state handl menu con...,perform issu regard enabl state handl menu con...,build identifi version build id i20120608 hi c...,2012-07-18 07:41:06,1.342610e+09,resolved fixed,8db6c32,1.389890e+09,bundles/org.eclipse.e4.ui.workbench.renderers....,bundles/org.eclipse.e4.ui.workbench.renderers....
2,2,3,423588,Bug 423588 [QuickAccess] Quick Access failure...,I was working in an inner from master.\n\nWhen...,quick access quickaccess quick access failur d...,quick access quickaccess quick access failur d...,work inner master restart workspac editor open...,2013-12-09 08:48:44,1.386600e+09,resolved fixed,6750d25,1.389820e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,bundles/org.eclipse.ui.workbench/EclipseUI/org...
3,3,4,420238,Bug 420238 [CSS] Reduce whitespace usage in t...,One of the major complains I hear is that the ...,cs reduc whitespac usag default theme one majo...,cs reduc whitespac usag default theme,one major complain hear new style wast lot spa...,2013-10-24 04:08:35,1.382600e+09,verified fixed,7710840,1.389800e+09,bundles/org.eclipse.e4.ui.workbench.renderers....,bundles/org.eclipse.e4.ui.workbench.renderers....
4,4,5,405216,Bug 405216 Widget disposed exception in Progr...,I have a job that sets the IProgressConstants....,widget dispos except progress info item progre...,widget dispos except progress info item progre...,job set progress constant iprogressconst actio...,2013-04-08 18:21:10,1.365460e+09,resolved fixed,470f274,1.389720e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,bundles/org.eclipse.ui.workbench/EclipseUI/org...


In [21]:
df_ui_nospace.to_csv('eclipse_platform_ui_noSpace.csv')